In [ ]:
detr_path = '/home/sworkspace/results/ab_car_validation_germany/riders/Co-DETR/2w_misses_codetr/co_detr.tsv'

In [ ]:
import glob
import pandas as pd
import pickle
import os

pickles_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/riders/Co-DETR/all/backlog/Co-DETR/raw_detections/pickles'
output_path = '/home/ubuntu/workspace/results/ab_car_validation_germany/riders/Co-DETR/all/backlog/Co-DETR'

def parse_pickles_to_dataframe():
    data = []
    for path in glob.iglob(f"{pickles_path}/*.pickle"):
        with open(path, 'rb') as file:
            detections = pickle.load(file)
            for detection in detections:
                bbox_str = str(detection.bbox)
                coords_str = bbox_str[bbox_str.index('<')+1:bbox_str.index('>')].split(',')[0:4]
                x1, y1, x2, y2 = map(float, [coord.strip("()") for coord in coords_str])
                x_center = (x1 + x2) / 2
                y_center = (y1 + y2) / 2
                width = x2 - x1
                height = y2 - y1
                label = detection.category.id
                if label in [0, 1, 2, 3, 5, 7]:
                    if label in [1, 3]:
                        label = 1
                    elif label in [2, 5, 7]:
                        label = 2
                    score = detection.score.value*100
                    filename = os.path.basename(path)
                    data.append((filename.replace('.pickle', '.png'), x_center, y_center, width, height, label, score))
                    
    df = pd.DataFrame(data, columns=['name', 'x_center', 'y_center', 'width', 'height', 'label', 'score'])
    df[['is_occluded', 'is_truncated', 'd3_separation', 'is_rider_on_2_wheels']] = 0
    df[['l_label', 'r_label']] = 'None'
    df['threshold'] = 'None'
    df.to_csv(f"{output_path}/co_detr.tsv", sep='\t',index=False)
    print(f"{output_path}/co_detr.tsv")

In [ ]:
parse_pickles_to_dataframe()

In [ ]:
import pandas as pd

In [ ]:
detr = pd.read_csv(f"{output_path}/co_detr.tsv", sep='\t')
detr['label'].value_counts()

In [ ]:
det = pd.read_csv('/home/ubuntu/workspace/results/ab_car_validation_germany/docker/2023_07_26__20_32/b2b_det.tsv', sep='\t')
det['label'].value_counts()